Momentum conversion
===================

Nomenclature
------------

All experimental geometry can be classified into two configurations, Type 1 and Type 2,
based on the relative position of the rotation axis and the analyzer slit. These can be
further divided into 4 configurations depending on the use of photoelectron deflectors
(DA).

Definition of angles differ for each geometry, but in all cases, $\delta$ is the
azimuthal angle that indicates in-plane rotation, $\alpha$ is the angle detected by
the analyzer, and $\beta$ is the angle along which mapping is performed.

For instance, imagine a typical Type 1 setup with a vertical slit that acquires maps by
rotating about the `z` axis in the lab frame. In this case, the polar angle (rotation
about `z`) is $\beta$, and the tilt angle becomes $\xi$.

The following table summarizes angle conventions for commonly encountered
configurations.

In [ ]:
import erlab.plotting as eplt
import matplotlib.pyplot as plt

In [ ]:
import xarray
# Before starting, we set some options for xarray.

# - The first option, ``display_expand_data=False``, will collapse the data by
#   default in the `repr` of DataArrays and Datasets.

# - The second option, ``keep_attrs=True``, ensures that the attributes of the data
#   are kept when performing operations such as sum and mean. Since parameters for
#   momentum conversion such as angle offsets and inner potential are stored as
#   attributes, this is important.
%config InlineBackend.figure_formats = ["svg", "pdf"]
plt.rcParams["figure.dpi"] = 96


_ = xarray.set_options(display_expand_data=False, keep_attrs=True)

Let's generate some example data, this time in angle coordinates.

In [ ]:
from erlab.io.exampledata import generate_data_angles

dat = generate_data_angles(shape=(200, 60, 300), assign_attributes=True, seed=1).T
dat

Let us define a 2D cut from the map data we just generated.

In [ ]:
cut = dat.sel(beta=10.0, method="nearest")
eplt.plot_array(cut)

Converting to momentum space
----------------------------

In [ ]:
dat_kconv = dat.kspace.convert()
dat_kconv

Let us plot the original and converted data side by side.

In [ ]:
fig, axs = plt.subplots(1, 2, layout="compressed")
eplt.plot_array(dat.sel(eV=-0.3, method="nearest"), ax=axs[0], aspect="equal")
eplt.plot_array(dat_kconv.sel(eV=-0.3, method="nearest"), ax=axs[1], aspect="equal")

Setting parameters
------------------

In [ ]:
dat.kspace.offsets

Since we haven't set any offsets, they are all zero. We will set the azimuthal angle to
60 degrees and the polar offset to 30 degrees and see what happens.

In [ ]:
dat.kspace.offsets.update(delta=60.0, beta=30.0)

In [ ]:
dat_kconv = dat.kspace.convert()
dat_kconv

Plotting the converted data again, we can see the effect of angle offsets on the conversion.

In [ ]:
fig, axs = plt.subplots(1, 2, layout="compressed")
eplt.plot_array(dat.sel(eV=-0.3, method="nearest"), ax=axs[0], aspect="equal")
eplt.plot_array(dat_kconv.sel(eV=-0.3, method="nearest"), ax=axs[1], aspect="equal")

Interactive conversion
----------------------